PROJETO 3 - CIÊNCIA DOS DADOS

Grupo: Gabriel Moura, Felipe Aron e Guilherme Aliperti


Projeto de regressão linear utilizando base de dados dos jogadores do FIFA. Pretendemos encontrar o coeficiente de determinação, simbolizado por r^2, que indica quanto da variação total é comum aos elementos que constituem os pares analisados. Analisaremos o coeficiente de terminação entre o valor do jogador com suas outras características.

In [64]:
import math
import os.path
import pandas as pd
import json
import numpy as np
from matplotlib import pyplot as plt
from random import shuffle
from scipy.stats import linregress
import statsmodels.api as sm

In [65]:
#Transforma os valores em float e sem os caracteres invalidos
df_completo = pd.read_excel("dados_completo.xls")
lista_final_valor = []
lista_final_sal = []

    #Ajustando os Valores dos jogadores para float...
for valores in df_completo["Value"]:
    valores = str(valores)
    l = valores.split("¬")
    del l[0]
    for x in l:
        carac = x.split("M")
        if len(carac) == 2:
            vf = float(carac[0]) * 1000000
            lista_final_valor.append(vf)
        else:
            carac = x.split("K")
            vf = float(carac[0]) * 1000
            lista_final_valor.append(vf)
            
# Ajustando os salários dos jogadores para float...          
for valores in df_completo["Wage"]:
    valores = str(valores)
    l = valores.split("¬")
    del l[0]
    for x in l:
        carac = x.split("M")
        if len(carac) == 2:
            vf = float(carac[0]) * 1000000
            lista_final_sal.append(vf)
        else:
            carac = x.split("K")
            vf = float(carac[0]) * 1000
            lista_final_sal.append(vf)
            
# Cria o Novo Dataframe ajustado...
df_valor_idade = pd.DataFrame({"Valor": lista_final_valor, "Overall": df_completo.Overall, "Idade": df_completo.Age,"Salário": lista_final_sal, "Potencial":df_completo.Potential})


Vamos analisar o coeficiente de determinação do valor em relação ao seu Overall por dois métodos diferentes:

In [66]:
# R2 do Overall em relação ao Valor

def lin_regression(x, y):
    m, b, R, p, SEm = linregress(x, y)

    n = len(x)
    SSx = np.var(x, ddof=1) * (n-1)  
    SEb2 = SEm **2 * (SSx/n + np.mean(x)**2)
    SEb = SEb2**0.5

    return m, b, SEm, SEb, R, p

m, b, Sm, Sb, R, p = lin_regression(lista_final_valor, df_completo.Overall)

print('m = {:>.4g} +- {:6.4f}'.format(m, Sm))
print('b = {:>.4g} +- {:6.4f}\n'.format(b, Sb))

print('R2 = {:7.5f}'.format(R**2))
print('p of test F : {:<8.6f}'.format(p))



m = 8.171e-07 +- 0.0000
b = 64.3 +- 0.0445

R2 = 0.39188
p of test F : 0.000000


In [67]:
df_valor_idade.describe()

,Idade,Overall,Potencial,Salário,Valor
count,17981.000000,17981.000000,17981.000000,17981.000000,1.798100e+04
mean,25.144541,66.247984,71.190813,11546.966242,2.385390e+06
std,4.614272,6.987965,6.102199,23080.000139,5.353970e+06
min,16.000000,46.000000,46.000000,0.000000,0.000000e+00
25%,21.000000,62.000000,67.000000,2000.000000,3.000000e+05
50%,25.000000,66.000000,71.000000,4000.000000,6.750000e+05
75%,28.000000,71.000000,75.000000,12000.000000,2.100000e+06
max,47.000000,94.000000,94.000000,565000.000000,1.230000e+08


In [68]:
# R2 do Overall em relação ao Valor
Y = df_valor_idade.Valor
X = df_valor_idade.Overall
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
results.summary()
results.rsquared


0.39187667210821164

Vamos agora avaliar o coeficiente entre o valor e a idade do jogador:

In [69]:
# R2 da Idade em relação ao Valor
Y = df_valor_idade.Valor
X = df_valor_idade.Idade
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
results.summary()
results.rsquared

0.0062899951432842105

O salário de um jogador geralmente tem muita relação com o seu valor no mercado no mundo real (sem contar os outliers, que seriam os altíssimos salários pagados por clubes chinenes e árabes), portanto é de se imaginar que o coeficiente de determinação entre o salário e o valor de um jogador é relativamente alto, portanto vamos analisar a inflûencia do salário de cada jogador no valor dele:

In [70]:
# R2 do Salário em relação ao Valor
Y = df_valor_idade.Valor
X = df_valor_idade.Salário
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
results.summary()
results.rsquared

0.72336759331866829

Vamos agora analisar o coeficiente de determinação de tres carcterísticas juntas (o salário, o potencial e o overall) em relação ao seu valor:

In [71]:
# R2 do Overall,Potencial e Salário juntos em relação ao Valor
Y = df_valor_idade.Valor
X = df_valor_idade.loc[:, ("Overall", "Potencial","Salário")]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
results.summary()
results.rsquared

0.75737551312652207

Até agora analisamos as características 

In [72]:
#Lista Atributos dos jogadores
lista_atributos = list(df_completo)
del lista_atributos[0:12]
del lista_atributos[34:62]

# RSquared Dicionário
rsquared = {}

#Ajustando Somas e Subtrações no Excel
for atributo in lista_atributos:
    lista = []
    for numeros in df_completo[atributo]:
        n = eval(str(numeros))
        lista.append(n)  
# R2 de cada Atributo                
    df_completo[atributo] = lista
    Y = df_valor_idade.Valor
    X = df_completo[atributo]
    X = sm.add_constant(X)
    model = sm.OLS(Y,X)
    results = model.fit()
    results.summary()
    rsquared[atributo] = results.rsquared

SyntaxError: invalid token (<string>, line 1)

In [ ]:
print("O atributo com maior R-Squared é: {0}".format(max(rsquared, key=rsquared.get)))
del rsquared[max(rsquared, key=rsquared.get)]
print("O segundo atributo com maior R-Squared é: {0}".format(max(rsquared, key=rsquared.get)))
del rsquared[max(rsquared, key=rsquared.get)]
print("O terceiro atributo com maior R-Squared é: {0}".format(max(rsquared, key=rsquared.get)))
del rsquared[max(rsquared, key=rsquared.get)]
print("O quarto atributo com maior R-Squared é: {0}".format(max(rsquared, key=rsquared.get)))
del rsquared[max(rsquared, key=rsquared.get)]
print("O quinto atributo com maior R-Squared é: {0}".format(max(rsquared, key=rsquared.get)))


In [ ]:
def atributos(a,b,c):
    Y = df_valor_idade.Overall
    X = df_completo.loc[:, (str(a),str(b),str(c))]
    X = sm.add_constant(X)
    model = sm.OLS(Y,X)
    results = model.fit()
    return results.rsquared







In [ ]:
import itertools
g = list(itertools.combinations(lista_atributos,3))

dicfinal = {}

for x in g:
    dicfinal[x[0]+" "+ x[1]+" "+x[2]] = atributos(x[0],x[1],x[2])

print("A combinação com maior R-Squared é: {0}".format(max(dicfinal, key=dicfinal.get)))

    

        